In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from math import sqrt
from datetime import datetime, timedelta

In [2]:
TRAIN_DATASET_FILE = 'data/sales_train.csv.gz'
ITEM_INFO_FILE = 'data/items.csv'

In [3]:
raw_train_set_df = pd.read_csv(TRAIN_DATASET_FILE)

In [4]:
raw_train_set_df["datetime"] = list(map(lambda d: datetime.strptime(d, '%d.%m.%Y'), raw_train_set_df["date"]))

In [5]:
print(raw_train_set_df.shape)
raw_train_set_df.tail()

(2935849, 7)


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,datetime
2935844,10.10.2015,33,25,7409,299.0,1.0,2015-10-10
2935845,09.10.2015,33,25,7460,299.0,1.0,2015-10-09
2935846,14.10.2015,33,25,7459,349.0,1.0,2015-10-14
2935847,22.10.2015,33,25,7440,299.0,1.0,2015-10-22
2935848,03.10.2015,33,25,7460,299.0,1.0,2015-10-03


In [6]:
min_date = min(raw_train_set_df.datetime)
max_date = max(raw_train_set_df.datetime)
delta = max_date - min_date
date_list = [min_date + timedelta(days=x) for x in range(0, delta.days+1)]
all_datetime_df = pd.DataFrame({"datetime": date_list})
print(all_datetime_df.shape)
all_datetime_df.head()

(1034, 1)


,datetime
0,2013-01-01
1,2013-01-02
2,2013-01-03
3,2013-01-04
4,2013-01-05


In [7]:
item_df = pd.read_csv(ITEM_INFO_FILE)

In [8]:
item_df.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [9]:
all_datetime_df["key"] = 1
item_df["key"] = 1

In [10]:
item_df.head()

,item_name,item_id,item_category_id,key
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40,1
1,!ABBYY FineReader 12 Professional Edition Full...,1,76,1
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,1
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,1
4,***КОРОБКА (СТЕКЛО) D,4,40,1


In [25]:
print(all_datetime_df.shape[0]*item_df.shape[0])

22923780


In [26]:
all_date_item_cat = all_datetime_df.merge(item_df[["key", "item_id", "item_category_id"]], on=["key"])
print(all_date_item_cat.shape)
all_date_item_cat.head()

(22923780, 4)


,datetime,key,item_id,item_category_id
0,2013-01-01,1,0,40
1,2013-01-01,1,1,76
2,2013-01-01,1,2,40
3,2013-01-01,1,3,40
4,2013-01-01,1,4,40


In [20]:
bigest_shop = raw_train_set_df[raw_train_set_df.item_cnt_day > 0].groupby("shop_id").item_cnt_day.sum().sort_values()
bigest_shop.tail()

shop_id
42    145195.0
28    184777.0
54    186149.0
25    242237.0
31    311230.0
Name: item_cnt_day, dtype: float64

#### We gonna create the dataframe for the biggest shop

In [29]:
all_date_item_cat[["datetime", "item_id", "item_category_id"]].drop_duplicates().shape

(22923780, 3)

In [30]:
all_date_item_cat[["datetime", "item_id"]].drop_duplicates().shape

(22923780, 2)

In [31]:
print(shop_31_data[["datetime", "item_id"]].drop_duplicates().shape)
print(shop_31_data[["datetime", "item_id"]].shape)

(235630, 2)
(235636, 2)


In [49]:
count_item_dt = shop_31_data[["datetime", "item_id", "shop_id"]]\
        .groupby(["datetime", "item_id"], as_index=False)["shop_id"].count()
count_item_dt[count_item_dt.shop_id > 1]

,datetime,item_id,shop_id
6750,2013-01-25,14050,2
116319,2014-03-09,12133,2
120965,2014-03-30,16875,2
148697,2014-08-14,9842,2
169551,2014-11-21,16587,2
177704,2014-12-22,8237,2


In [47]:
shop_31_data[(shop_31_data.datetime == "2014-03-09") & (shop_31_data.item_id == 12133)]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,datetime
1507628,09.03.2014,14,31,12133,399.0,1.0,2014-03-09
1507629,09.03.2014,14,31,12133,599.0,1.0,2014-03-09


In [52]:
shop_31_data = raw_train_set_df[raw_train_set_df.shop_id == 31]
full_shop_31_data_with_na = all_date_item_cat[["datetime", "item_id", "item_category_id"]].merge(\
        shop_31_data[["date_block_num", "item_id", "item_price", "item_cnt_day", "datetime"]],
        on=["datetime", "item_id"], how="outer")

In [53]:
print(all_date_item_cat.shape[0])
print(full_shop_31_data_with_na.shape[0])
full_shop_31_data_with_na.head()

22923780
22923786


,datetime,item_id,item_category_id,date_block_num,item_price,item_cnt_day
0,2013-01-01,0,40,NaN,NaN,NaN
1,2013-01-01,1,76,NaN,NaN,NaN
2,2013-01-01,2,40,NaN,NaN,NaN
3,2013-01-01,3,40,NaN,NaN,NaN
4,2013-01-01,4,40,NaN,NaN,NaN


In [55]:
full_shop_31_data_with_na = full_shop_31_data_with_na.groupby(["datetime", "item_id", "item_category_id"],
                                                              as_index=False).agg(
                                                                {
                                                                    "item_price": "max",
                                                                    "item_cnt_day": "sum"
                                                                })
print(full_shop_31_data_with_na.shape)

(22923780, 5)


In [57]:
full_shop_31_data_with_na.fillna_cnt_day"] = full_shop_31_data_with_na["item_cnt_day"].fillna(0)
full_shop_31_data_with_na.head()

,datetime,item_id,item_category_id,item_price,item_cnt_day
0,2013-01-01,0,40,NaN,0.0
1,2013-01-01,1,76,NaN,0.0
2,2013-01-01,2,40,NaN,0.0
3,2013-01-01,3,40,NaN,0.0
4,2013-01-01,4,40,NaN,0.0


In [59]:
full_shop_31_data_with_na.to_csv('data/save_of_shop_31_not_finished_yet.csv.gz', compression='gzip')

### We need to fill the nan value in price by the average price in the shop

To do so we will need to check is the shop sell the item

### Now we need to do the sum over 31 previous days 